In [47]:
import numpy as np
import pandas as pd
import json
import os
import re


## USA dataset

In [48]:
# get all json file path in the folder
folder_path = "../dataset/USA"

file_paths = []
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        file_paths.append(filepath)

len(file_paths)

47

In [49]:
# load json file
job_results_in_US = []

for file_path in file_paths:
    with open(file_path, 'r') as f:
        data = json.load(f)
    job_results_in_US.append(data)

len(job_results_in_US)

47

In [50]:
jobs_data = pd.DataFrame(columns=['title', 'company_name', 'location', 'via', 'description', 'qualifications', 'responsibilities', 'benefits', 'job_id'])

## Make dataframe

In [40]:
for dataset in job_results_in_US:
    if 'jobs_results' not in dataset:
        continue  
    jobs_results = dataset["jobs_results"]
    for job_result in jobs_results:
        title = job_result["title"]
        company_name = job_result["company_name"]
        location = job_result["location"] 
        via = job_result["via"] 
        description = job_result["description"] 
        job_highlights = job_result["job_highlights"] 

        qualifications = None 
        responsibilities = None 
        benefits = None 

        for highlight in job_highlights:
            if highlight.get("title") == "Qualifications":
                qualifications = highlight.get("items") 
            elif highlight.get("title") == "Responsibilities":
                responsibilities = highlight.get("items")
            elif highlight.get("title") == 'Benefits':
                benefits = highlight.get("items") 

        job_id = job_result["job_id"]
        data = {'title': title, 'company_name': company_name, 'location': location, 'via': via, 'description': description, 'qualifications': qualifications, 'responsibilities': responsibilities, 'benefits': benefits, 'job_id': job_id}
        jobs_data = jobs_data.append(data, ignore_index=True)


/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/162586353.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data = jobs_data.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/162586353.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data = jobs_data.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/162586353.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data = jobs_data.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/162586353.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version.

In [41]:
jobs_data.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,"[Language Fluency: English (Required), The req...",[Deliver highly complex solutions with signifi...,[General Description of Available Benefits for...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,None,None,None,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,[Ability to parse and understand EVM (and/or S...,[Tackling interesting problems - we are ingest...,[An accelerated growth trajectory - you will h...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",[We’re looking for a capable generalist engine...,[You will be designing and building our core s...,"[We’re wrapping up our presale for $JAM, so yo...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,None,None,None,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...


In [46]:
jobs_data.to_csv('../dataset/USA_dataset_nolat.csv', index=False)

In [42]:
subset_dataset = jobs_data.dropna(subset=['location'])
subset_dataset = subset_dataset[~subset_dataset['location'].str.contains('Anywhere|United States')]
pattern = r"\s*\([^)]*\)"
subset_dataset["location"] = subset_dataset["location"].str.replace(pattern, "", regex=True)
subset_dataset['via'] = subset_dataset['via'].str.replace('via ', '')

print(subset_dataset['location'].value_counts())

  New York, NY                 24
  San Francisco, CA            16
  Annapolis Junction, MD       12
  Chicago, IL                  11
  Palo Alto, CA                 9
                               ..
  Westborough, MA               1
  Downers Grove, IL             1
  Houston, TX                   1
  Trenton, NJ                   1
  Minneapolis, MN               1
Name: location, Length: 135, dtype: int64


In [43]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def geocode_location(location):
    try:
        geolocator = Nominatim(user_agent="my-custom-agent")
        location = geolocator.geocode(location)
        if location is None:
            return None
        else:
            return (location.latitude, location.longitude)
    except GeocoderTimedOut:
        return None

subset_dataset['location_coords'] = subset_dataset['location'].apply(geocode_location)
subset_dataset[['latitude', 'longitude']] = pd.DataFrame(subset_dataset['location_coords'].tolist(), index=subset_dataset.index)

In [44]:
subset_dataset.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,location_coords,latitude,longitude
2,Blockchain Analyst,Allium,"New York, NY",Jobs,About the role\n• Sherlock & Enola Holmes leve...,[Ability to parse and understand EVM (and/or S...,[Tackling interesting problems - we are ingest...,[An accelerated growth trajectory - you will h...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,"(40.7127281, -74.0060152)",40.712728,-74.006015
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",[We’re looking for a capable generalist engine...,[You will be designing and building our core s...,"[We’re wrapping up our presale for $JAM, so yo...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,"(34.0536909, -118.242766)",34.053691,-118.242766
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,None,None,None,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,"(37.6820583, -121.768053)",37.682058,-121.768053
5,Blockchain Data Analytics: Tech Lead - Assista...,EY,"New York, NY",EY Careers,"At EY, you’ll have the chance to build a caree...","[Agile Delivery and Development, Quality Assur...",[Technical leadership of data focused products...,[Other geographies will follow their local sal...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIERhdGEgQW5hbH...,"(40.7127281, -74.0060152)",40.712728,-74.006015
6,STI25526 Technical Architect Block Chain,Software Technology Inc.,"Irving, TX",LinkedIn,Title: Technical Architect Block Chain\n\nLoca...,[We need person with RUST (Block chain) knowle...,[Develop cutting-edge software solutions in ra...,None,eyJqb2JfdGl0bGUiOiJTVEkyNTUyNiBUZWNobmljYWwgQX...,"(32.8295183, -96.9442177)",32.829518,-96.944218


In [45]:
subset_dataset.to_csv('../dataset/USA_dataset.csv', index=False)

## DC dataset

In [51]:
# get all json file path in the folder
folder_path = "../dataset/DC"

file_paths = []
for filename in os.listdir(folder_path):
    filepath = os.path.join(folder_path, filename)
    if os.path.isfile(filepath):
        file_paths.append(filepath)

len(file_paths)


38

In [52]:
# load json file
job_results_in_DC = []

for file_path in file_paths:
    with open(file_path, 'r') as f:
        data = json.load(f)
    job_results_in_DC.append(data)

len(job_results_in_DC)

38

In [53]:
jobs_data2 = pd.DataFrame(columns=['title', 'company_name', 'location', 'via', 'description', 'qualifications', 'responsibilities', 'benefits', 'job_id'])

In [54]:
for dataset in job_results_in_DC:
    if 'jobs_results' not in dataset:
        continue  
    jobs_results = dataset["jobs_results"]
    for job_result in jobs_results:
        title = job_result["title"]
        company_name = job_result["company_name"]
        location = job_result["location"] 
        via = job_result["via"] 
        description = job_result["description"] 
        job_highlights = job_result["job_highlights"] 

        qualifications = None 
        responsibilities = None 
        benefits = None 

        for highlight in job_highlights:
            if highlight.get("title") == "Qualifications":
                qualifications = highlight.get("items") 
            elif highlight.get("title") == "Responsibilities":
                responsibilities = highlight.get("items")
            elif highlight.get("title") == 'Benefits':
                benefits = highlight.get("items") 

        job_id = job_result["job_id"]
        data = {'title': title, 'company_name': company_name, 'location': location, 'via': via, 'description': description, 'qualifications': qualifications, 'responsibilities': responsibilities, 'benefits': benefits, 'job_id': job_id}
        jobs_data2 = jobs_data2.append(data, ignore_index=True)


/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/2680677936.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data2 = jobs_data2.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/2680677936.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data2 = jobs_data2.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/2680677936.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jobs_data2 = jobs_data2.append(data, ignore_index=True)
/var/folders/h0/sq_brb3d0_lfvtk4fhlbrck00000gn/T/ipykernel_42185/2680677936.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [55]:
jobs_data2.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id
0,Ethereum Blockchain Developer (Remote),Ex Populus,Anywhere,via Built In,Company Overview:\nEx Populus is a cutting-edg...,"[2-3 years of Software Development experience,...","[Design, maintain and deploy smart contracts f...",None,eyJqb2JfdGl0bGUiOiJFdGhlcmV1bSBCbG9ja2NoYWluIE...
1,Blockchain Engineer,21.co,"New York, NY",via Greenhouse,We are seeking a highly motivated and skilled ...,[Bachelor's or Master's degree in Computer Sci...,"[As a Blockchain Engineer, you will be respons...",[(NYC only) Pursuant to Section 8-102 of title...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIi...
2,Blockchain Course Instructor,Blockchain Institute of Technology,Anywhere,via LinkedIn,"Are you a blockchain, cryptocurrency, NFT, Met...","[3+ years of experience in blockchain, cryptoc...",[Our expert technical team will provide the su...,None,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIENvdXJzZSBJbn...
3,Python based - Blockchain developer to join ex...,Upwork,Anywhere,via Upwork,Need someone to join our existing team to spee...,"[Candidates must be willing to sign, non-discl...",[Will discuss details with the selected candid...,None,eyJqb2JfdGl0bGUiOiJQeXRob24gYmFzZWQgLSBCbG9ja2...
4,Blockchain DevOps Engineer (Remote),Telnyx,United States,via Startup Jobs,"About Telnyx\n\nAt Telnyx, we’re architecting ...",[You are a highly motivated and experienced Bl...,[To build a best-in-class Filecoin (FIL) Minin...,None,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIERldk9wcyBFbm...


In [56]:
jobs_data2.to_csv('../dataset/DC_dataset.csv', index=False)

# Combine two datasets

In [18]:
combined_dataset = pd.concat([jobs_data, jobs_data2], ignore_index=True)
combined_dataset.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id
0,Senior Blockchain Software Engineer,Truist,Anywhere,via ZipRecruiter,The position is described below. If you want t...,"[Language Fluency: English (Required), The req...",[Deliver highly complex solutions with signifi...,[General Description of Available Benefits for...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgQmxvY2tjaGFpbiBTb2...
1,Get twitter accounts that are tied to blockcha...,Upwork,Anywhere,via Upwork,I am looking for a list of blockchain addresse...,None,None,None,eyJqb2JfdGl0bGUiOiJHZXQgdHdpdHRlciBhY2NvdW50cy...
2,Blockchain Analyst,Allium,"New York, NY",via Jobs,About the role\n• Sherlock & Enola Holmes leve...,[Ability to parse and understand EVM (and/or S...,[Tackling interesting problems - we are ingest...,[An accelerated growth trajectory - you will h...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",via AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",[We’re looking for a capable generalist engine...,[You will be designing and building our core s...,"[We’re wrapping up our presale for $JAM, so yo...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","via Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,None,None,None,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...


## Clean

In [19]:
# Drop duplicate
combined_dataset = combined_dataset.drop_duplicates(subset='job_id')
len(combined_dataset)

823

## Save file

In [20]:
combined_dataset.to_csv('../dataset/combined_dataset.csv', index=False)

In [21]:
print(combined_dataset['location'].value_counts())

 Anywhere                       177
  Washington, DC                 44
  New York, NY                   44
  United States                  33
  San Francisco, CA              28
                               ... 
  Cary, NC                        1
  Boston, MA  (+1 other)          1
  Deerfield, IL                   1
  Evanston, IL                    1
  Los Altos, CA                   1
Name: location, Length: 170, dtype: int64


In [22]:
subset_dataset = combined_dataset.dropna(subset=['location'])
subset_dataset = subset_dataset[~subset_dataset['location'].str.contains('Anywhere|United States')]
pattern = r"\s*\([^)]*\)"
subset_dataset["location"] = subset_dataset["location"].str.replace(pattern, "", regex=True)
subset_dataset['via'] = subset_dataset['via'].str.replace('via ', '')

print(subset_dataset['location'].value_counts())

  New York, NY                 45
  Washington, DC               44
  San Francisco, CA            28
  Annapolis Junction, MD       21
  Arlington, VA                19
                               ..
  Arlington, TX                 1
  Princeton, NJ                 1
  Huntsville, AL                1
  Illinois                      1
  Los Altos, CA                 1
Name: location, Length: 163, dtype: int64


## long and lat

In [23]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def geocode_location(location):
    try:
        geolocator = Nominatim(user_agent="my-custom-agent")
        location = geolocator.geocode(location)
        if location is None:
            return None
        else:
            return (location.latitude, location.longitude)
    except GeocoderTimedOut:
        return None

subset_dataset['location_coords'] = subset_dataset['location'].apply(geocode_location)
subset_dataset[['latitude', 'longitude']] = pd.DataFrame(subset_dataset['location_coords'].tolist(), index=subset_dataset.index)


In [25]:
subset_dataset.head()

,title,company_name,location,via,description,qualifications,responsibilities,benefits,job_id,location_coords,latitude,longitude
2,Blockchain Analyst,Allium,"New York, NY",Jobs,About the role\n• Sherlock & Enola Holmes leve...,[Ability to parse and understand EVM (and/or S...,[Tackling interesting problems - we are ingest...,[An accelerated growth trajectory - you will h...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEFuYWx5c3QiLC...,"(40.7127281, -74.0060152)",40.712728,-74.006015
3,Full Stack Blockchain Engineer,geojam,"Los Angeles, CA",AngelList,"Who We Are\n\nLaunched in 2020, Geojam is a so...",[We’re looking for a capable generalist engine...,[You will be designing and building our core s...,"[We’re wrapping up our presale for $JAM, so yo...",eyJqb2JfdGl0bGUiOiJGdWxsIFN0YWNrIEJsb2NrY2hhaW...,"(34.0536909, -118.242766)",34.053691,-118.242766
4,Blockchain Engineer - Apps Platform at Exodus ...,Exodus,"Livermore, CA","Livermore, CA - Geebo",Exodus is looking for a detail-orientated Bloc...,None,None,None,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIEVuZ2luZWVyIC...,"(37.6820583, -121.768053)",37.682058,-121.768053
5,Blockchain Data Analytics: Tech Lead - Assista...,EY,"New York, NY",EY Careers,"At EY, you’ll have the chance to build a caree...","[Agile Delivery and Development, Quality Assur...",[Technical leadership of data focused products...,[Other geographies will follow their local sal...,eyJqb2JfdGl0bGUiOiJCbG9ja2NoYWluIERhdGEgQW5hbH...,"(40.7127281, -74.0060152)",40.712728,-74.006015
6,STI25526 Technical Architect Block Chain,Software Technology Inc.,"Irving, TX",LinkedIn,Title: Technical Architect Block Chain\n\nLoca...,[We need person with RUST (Block chain) knowle...,[Develop cutting-edge software solutions in ra...,None,eyJqb2JfdGl0bGUiOiJTVEkyNTUyNiBUZWNobmljYWwgQX...,"(32.8295183, -96.9442177)",32.829518,-96.944218


In [26]:
subset_dataset.to_csv('../dataset/combined_dataset_long_lat.csv', index=False)